In [ ]:
!git clone -b v6.0 https://github.com/ultralytics/yolov5.git

In [ ]:
# 上传data.zip (yolo格式) 到yolov5并目录下
!unzip -q data.zip -d yolov5/data_up
!rm -rf yolov5/data/images
!mv yolov5/data_up/images yolov5/data/
!mv yolov5/data_up/labels yolov5/data/
!mv yolov5/data_up/train.txt yolov5/data/
!mv yolov5/data_up/test.txt yolov5/data/
!mv yolov5/data_up/val.txt yolov5/data/

#### 在yolov5目录下创建 model.yml 和 dataset.yml
- model.yml从yolov5/models/下面复制
- dataset.yml
```python
train: data/train.txt  # 数据集里的训练集列表文件路径
val: data/val.txt  # 数据集里的验证集列表文件路径
nc: 2 #类型数量
names: [ 'blue', 'red' ]  #类型名
```

#### 记得在yolov5/models/common.py最下面添加如下内容
- 不然会报错:AttributeError: Cant get attribute SPPF on module models.common
```python
import warnings
class SPPF(nn.Module):
    # Spatial Pyramid Pooling - Fast (SPPF) layer for YOLOv5 by Glenn Jocher
    def __init__(self, c1, c2, k=5):  # equivalent to SPP(k=(5, 9, 13))
        super().__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)
    def forward(self, x):
        x = self.cv1(x)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')  # suppress torch 1.9.0 max_pool2d() warning
            y1 = self.m(x)
            y2 = self.m(y1)
            return self.cv2(torch.cat([x, y1, y2, self.m(y2)], 1))


```

In [ ]:
import os
os.chdir('yolov5')

#!pip install -r requirements.txt
#!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
!python train.py --data dataset.yml --cfg model.yml --weights yolov5s.pt --device 0

os.chdir('../')

In [ ]:
# 查看识别效果
import os
os.chdir('yolov5')

!python detect.py --source data/images/0.jpg --weights runs/train/exp/weights/best.pt --view-img

os.chdir('../')

In [ ]:
!git clone https://github.com/wang-xinyu/tensorrtx.git

!mv tensorrtx/yolov5/gen_wts.py yolov5/

In [ ]:
# 转化生成.wts文件
import os
os.chdir('yolov5')

!python gen_wts.py --weight=runs/train/exp4/weights/best.pt --output=./

os.chdir('../')

In [ ]:
!pip install torchsummaryX

In [ ]:
#debug部分
#!mv best.pt yolov5/
import os
#os.chdir('yolov5')

import torch
from torchsummaryX import summary

from models.experimental import attempt_load

model1_path = r'best.pt'
#model2_path = r'D:\Install\yolov5l.pt'

model = attempt_load(model1_path, map_location=torch.device('cpu'))
# model.to('cuda').eval()
# print(model)
summary(model, torch.rand((1, 3, 640, 640)))

os.chdir('../')